In [92]:
import os, glob
import pandas as pd
from IPython.core import display as ICD 
import numpy as np
pd.set_option('display.max_rows', 60)


In [138]:
def make_bids_report(data_dict,df):
    report_df = pd.read_csv('/projects/niblab/bids_projects/Experiments/bro/bids/code/qc_report_s1.csv', index_col='subject')
    #ICD.display(report_df.head())
    #print(df.columns.values)
    #ICD.display(df.loc[:, ['func_ct_v1', 'anat_ct_v1','fmap_ct_v1']])
    
    for sub_id in report_df.index.values:
            
        try:
            # check BIDS 
            func_ct = df.loc[sub_id, "func_ct_v1"]
            anat_ct = df.loc[sub_id, "anat_ct_v1"]
            fmap_ct = df.loc[sub_id, "fmap_ct_v1"]

            if func_ct == 5 and anat_ct == 1 and fmap_ct == 3:
                report_df.loc[sub_id, "BIDS check"] = "pass"

            elif np.isnan(func_ct) == True and np.isnan(anat_ct) == True and np.isnan(anat_ct) == True:
                report_df.loc[sub_id, "BIDS check"] = "no files found"

            elif np.isnan(func_ct) == True and np.isnan(anat_ct) != True and np.isnan(anat_ct) != True:
                report_df.loc[sub_id, "BIDS check"] = "missing funcs"

            elif np.isnan(func_ct) != True and np.isnan(anat_ct) == True and np.isnan(anat_ct) != True:
                report_df.loc[sub_id, "BIDS check"] = "missing anat"

            elif np.isnan(func_ct) != True and np.isnan(anat_ct) != True and np.isnan(anat_ct) == True:
                report_df.loc[sub_id, "BIDS check"] = "missing fmap"

            else:
                report_df.loc[sub_id, "BIDS check"] = "missing files"
        except:
            report_df.loc[sub_id, "BIDS check"] = "missing files"
            
        
            
            
    ICD.display(report_df)
    ICD.display(df)
    

In [139]:
def bids_report():
    bids_dir = '/projects/niblab/bids_projects/Experiments/bro/bids'
    bids_subs = sorted(glob.glob(os.path.join(bids_dir, "sub-*")))
    bids_dict ={}
    sess_dict = {}
    
    
    ses1_bids = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids", "sub-*/ses-1"))]
    ses2_bids = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids", "sub-*/ses-2"))]
    
    sess_1_ct = len(ses1_bids)
    sess_2_ct = len(ses2_bids)
    
    ses1_dcms = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM", "sub-*/ses-1"
))]
    ses2_dcms = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM", "sub-*/ses-2"
))]
    
    
    mia_bids_s1 = [x for x in ses1_dcms if x not in ses1_bids]
    mia_bids_s2 = [x for x in ses2_dcms if x not in ses2_bids]



    for path in bids_subs:
        sub_id = path.split("/")[-1]
        if sub_id not in bids_dict:
            bids_dict[sub_id] = {}
            
        sessions = glob.glob(os.path.join(path,"ses-*"))
        
        for sess_path in sessions:
            
            # initialize and increment variables
            
            sess_id = sess_path.split("/")[-1].split("-")[1]
            
            if sess_id not in sess_dict:
                sess_dict[sess_id] = {}
                
            #sess_dict[sess_id]["sub_count"] = sess_ct  
            
            
            funcs = glob.glob(os.path.join(sess_path, "func/*.nii.gz"))
            anats = glob.glob(os.path.join(sess_path, "anat/*.nii.gz"))
            fmaps = glob.glob(os.path.join(sess_path, "fmap/*.nii.gz"))
            
            col_func_name = "func_ct_v{}".format(sess_id)
            col_anat_name = "anat_ct_v{}".format(sess_id)
            col_fmap_name = "fmap_ct_v{}".format(sess_id)
            
            #print(col_func_name)
            
            bids_dict[sub_id][col_func_name] = int(len(funcs))
            bids_dict[sub_id][col_anat_name] = len(anats)
            bids_dict[sub_id][col_fmap_name] = len(fmaps)
            bids_dict[sub_id][col_fmap_name] = len(fmaps)
            #print(funcs, anats, fmaps)
    
    
    df=pd.DataFrame.from_dict(bids_dict).T
    df.index.name = "participantID"
    
    make_bids_report(bids_dict, df)

    #ICD.display(df)
    
    null_df = df[df.isnull().any(axis=1)]
    
    """print("\nSession 1 bids subject count:", sess_1_ct )
    print("Subject ids: \n{}\n".format([x.split("/")[-2] for x in glob.glob(os.path.join(bids_dir, "sub-*","ses-1"))]))
    
    if not mia_bids_s1:
        print('passing')
    else:    
        print("Raw DICOMS ready to be processed: ", mia_bids_s1)
    
    print("Session 2 bids subject count:", sess_2_ct )
    print("Subject ids: \n{}\n".format([x.split("/")[-2] for x in glob.glob(os.path.join(bids_dir, "sub-*","ses-2"))]))

    print("\nSubjects with missing files:")
    ICD.display(null_df)
    
    print("\nAll subject file counts:")
    ICD.display(df)"""
    
    return ses1_bids, ses2_bids;


In [140]:
def fmriprep(s1_bids, s2_bids):
    fmriprep_dir = "/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/fmriprep"
    fprep_subs = [x for x in sorted(glob.glob(os.path.join(fmriprep_dir, "sub-*"))) if ".html" not in x]
    data_dict ={}
    sess_dict = {}
    
        
    ses1_fprep = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/fmriprep", "sub-*/ses-1"))]
    ses2_fprep = [x.split("/")[-2] for x in glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/fmriprep", "sub-*/ses-2"))]
    
        
    mia_s1 = [x for x in s1_bids if x not in ses1_fprep]
    mia_s2 = [x for x in s2_bids if x not in ses2_fprep]

    for path in fprep_subs:
        sub_id = path.split("/")[-1]
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
        #print(sub_id)
        
        anat = glob.glob(os.path.join(path, "anat/*desc-preproc_T1w.nii.gz"))
        #print(anat)
        data_dict[sub_id]["anat_ct"] = len(anat)

        sessions = glob.glob(os.path.join(path,"ses-*"))
        
        for sess_path in sessions:
            #print(sess_path)
        
            # initialize and increment variables

            sess_id = sess_path.split("/")[-1].split("-")[1]

            if sess_id not in sess_dict:
                sess_dict[sess_id] = {}

            #sess_dict[sess_id]["sub_count"] = sess_ct  


            funcs = glob.glob(os.path.join(sess_path, "func/*_desc-preproc_bold.nii.gz"))
            #fmaps = glob.glob(os.path.join(sess_path, "fmap/*.nii.gz"))

            col_func_name = "func_ct_s{}".format(sess_id)
            #col_anat_name = "anat_ct_v{}".format(sess_id)
            #col_fmap_name = "fmap_ct_v{}".format(sess_id)

            #print(col_func_name)

            data_dict[sub_id][col_func_name] = int(len(funcs))
            #bids_dict[sub_id][col_fmap_name] = len(fmaps)
            #bids_dict[sub_id][col_fmap_name] = len(fmaps)
            #print(funcs, anats, fmaps)"""
        
    
    df=pd.DataFrame.from_dict(data_dict).T
    df.index.name = "participantID"
    
        
    null_df = df[df.isnull().any(axis=1)]
    
    print("\nSession 1 \n")
    print("fmriprep subject count:", len(ses1_fprep) )
    #print("Subject ids: \n{}\n".format([x.split("/")[-2] for x in glob.glob(os.path.join(bids_dir, "sub-*","ses-1"))]))
    
    if not mia_s1:
        print('\npassing')
    else:    
        print("\nbids subjects ready to be processed: \n", mia_s1)
    
    print("\nSession 2 \n")
    print("fmriprep subject count:",  len(ses2_fprep) )
    #print("Subject ids: \n{}\n".format([x.split("/")[-2] for x in glob.glob(os.path.join(bids_dir, "sub-*","ses-2"))]))
    
    if not mia_s2:
        print('\npassing')
    else:    
        print("\nbids subjects ready to be processed: \n", mia_s2)
        
        
    print("\nSubjects with missing files:")
    ICD.display(null_df)
    
    #print("\nAll subject file counts:")
    #ICD.display(df)
    
    ICD.display(df)

    

## BIDS data

In [141]:
s1_bids, s2_bids = bids_report()


,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,sdc_r4,From TSV Files,fd_r1,min_r1,max_r1,mcflirt_3mm_r1,fd_r2
subject,,,,,,,,,,,,,,,,,
sub-001,no files found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,no files found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,missing files,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-006,missing files,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-007,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-008,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-009,pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,func_ct_v2,anat_ct_v2,fmap_ct_v2,func_ct_v1,anat_ct_v1,fmap_ct_v1
participantID,,,,,,
sub-001,5.0,1.0,3.0,NaN,NaN,NaN
sub-002,5.0,1.0,3.0,NaN,NaN,NaN
sub-003,4.0,0.0,3.0,5.0,1.0,3.0
sub-004,5.0,1.0,3.0,5.0,1.0,3.0
sub-006,5.0,1.0,3.0,5.0,1.0,0.0
sub-007,5.0,1.0,3.0,5.0,1.0,3.0
sub-008,0.0,1.0,1.0,5.0,1.0,3.0
sub-009,5.0,1.0,3.0,5.0,1.0,3.0
sub-010,5.0,1.0,3.0,0.0,1.0,1.0


## fMRIprep data

In [11]:
fmriprep(s1_bids, s2_bids)


Session 1 

fmriprep subject count: 32

bids subjects ready to be processed: 
 ['sub-003', 'sub-004', 'sub-006', 'sub-012', 'sub-013', 'sub-016', 'sub-017', 'sub-018', 'sub-023', 'sub-031', 'sub-034', 'sub-037', 'sub-039', 'sub-040', 'sub-045', 'sub-048', 'sub-054', 'sub-055']

Session 2 

fmriprep subject count: 35

bids subjects ready to be processed: 
 ['sub-003', 'sub-006', 'sub-012', 'sub-013', 'sub-016', 'sub-017', 'sub-018', 'sub-023', 'sub-031', 'sub-034', 'sub-039', 'sub-040', 'sub-042', 'sub-045', 'sub-048', 'sub-055']

Subjects with missing files:


,anat_ct,func_ct_s2,func_ct_s1
participantID,,,
sub-001,2.0,5.0,NaN
sub-002,2.0,5.0,NaN
sub-004,2.0,5.0,NaN
sub-012,0.0,NaN,NaN
sub-020,2.0,NaN,3.0
sub-027,2.0,5.0,NaN
sub-035,0.0,NaN,0.0
sub-054,2.0,5.0,NaN


,anat_ct,func_ct_s2,func_ct_s1
participantID,,,
sub-001,2.0,5.0,NaN
sub-002,2.0,5.0,NaN
sub-004,2.0,5.0,NaN
sub-007,2.0,0.0,0.0
sub-008,2.0,0.0,5.0
sub-009,2.0,5.0,5.0
sub-010,2.0,5.0,0.0
sub-011,2.0,5.0,5.0
sub-012,0.0,NaN,NaN


## Preprocessing 